In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
images_folder = "path/to/images_folder"  
csv_path = "path/to/annotations.csv"     
df = pd.read_csv(csv_path)

In [ ]:
def image_exists(image_name):
    return os.path.isfile(os.path.join(images_folder, image_name))

In [ ]:
df = df[df['image_name'].apply(image_exists)]
annotated_images = set(df['image_name'].unique())
all_images = set(os.listdir(images_folder))
negative_images = list(all_images - annotated_images)

In [ ]:
IMG_SIZE = 224
data = []    
labels = []   

In [ ]:
for idx, row in df.iterrows():
    img_path = os.path.join(images_folder, row['image_name'])
    img = cv2.imread(img_path)
    if img is None:
        continue  
    x0, y0, x1, y1 = int(row['x0']), int(row['y0']), int(row['x1']), int(row['y1'])
    face_img = img[y0:y1, x0:x1]
    if face_img.size == 0:
        continue 
    face_img = cv2.resize(face_img, (IMG_SIZE, IMG_SIZE))
    face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
    face_img = face_img.astype("float32") / 255.0
    data.append(face_img)
    labels.append(1)  

In [ ]:
for image_name in negative_images:
    img_path = os.path.join(images_folder, image_name)
    img = cv2.imread(img_path)
    if img is None:
        continue
    img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img_resized = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
    img_resized = img_resized.astype("float32") / 255.0
    data.append(img_resized)
    labels.append(0)  

In [ ]:
x = np.array(data)
y = np.array(labels)

# Разбиваем данные на обучающую и валидационную выборки (80% - 20%).
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))


for layer in base_model.layers:
    layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

output = Dense(1, activation='sigmoid', name='face_output')(x)


model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=10, batch_size=32)


model.save("trained_face_detection_model.h5")